# Citi Bike Manhattan Filtering

---
Last Edited : 2023/12/10 by Leonard Tsai

# 1. Filter Manhanttan Data

In [2]:
import os
import glob
import pandas as pd
import geopandas as gpd

# Define your functions for processing each file
def process_csv(csv_file):
    selected_columns = ['started_at', 'start_lat', 'start_lng']
    # Manhattan polygon boundaries 
    manhattan = gpd.read_file("manhattan-island.geojson")

    _df = pd.read_csv(csv_file)
    df = _df[selected_columns]
    del _df

    # Convert 'started_at' to datetime
    df['started_at'] = pd.to_datetime(df['started_at'])
    # Add 9 minutes to each entry
    df['started_at'] = df['started_at'] + pd.to_timedelta('9 minutes')
    # Convert 'started_at' to the desired format
    df['started_at'] = df['started_at'].dt.strftime('%Y%m%d %H')

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['start_lng'], df['start_lat']))
    gdf.crs = 'EPSG:4326'
    gdf = gdf[['started_at', 'geometry']]
    del df

    gdf_filtered = gpd.sjoin(gdf, manhattan, how='inner', op='within').reset_index(drop=True)
    gdf_filtered = gdf_filtered[['started_at', 'geometry']]

    # Get the filename without the extension
    file_name = os.path.splitext(os.path.basename(csv_file))[0]
    result_df = gdf_filtered.groupby('started_at').size().reset_index(name='count')
    result_df.to_csv(f"./202211-202310/{file_name}_filtered.csv")
    print(f"Saved {file_name} successfully!")

In [5]:
csv_file = "../original_data/202311-citibike-tripdata.csv"
process_csv(csv_file)

/var/folders/k2/_l426jsn7b5gddd0ylpgz4nh0000gn/T/ipykernel_18213/2546030723.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  _df = pd.read_csv(csv_file)
/Users/leonardtsai/miniconda3/envs/tensorflow/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Saved 202311-citibike-tripdata successfully!


# 2. Concatenate all data and weather data

In [6]:
weather_df = pd.read_csv("./202211-202310/new_weather_20231210.csv")

In [7]:
folder_path = './202211-202310'
file_pattern = '*.csv'

# Use glob to find files matching the pattern
filtered_csv_files = sorted(glob.glob(os.path.join(folder_path, file_pattern)))
filtered_csv_files.remove('./202211-202310/new_weather_20231210.csv')

filtered_df = pd.concat([pd.read_csv(filtered_csv) for filtered_csv in filtered_csv_files], ignore_index=True).groupby('started_at')['count'].sum().reset_index().iloc[:-1]

In [8]:
# Merge DataFrames based on the common columns
merged_df = pd.merge(weather_df, filtered_df, left_on='Date', right_on='started_at', how='inner')

# Drop the duplicate column (you can customize this based on your needs)
merged_df = merged_df.drop('started_at', axis=1)

In [9]:
merged_df.to_csv("merged_weather.csv", index=False)